In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))


X_train = pd.read_json("../input/train.json")
X_test = pd.read_json("../input/test.json")

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.model_selection import train_test_split
X_train.dropna(subset = ['interest_level'])
X_train["num_photos"] = X_train["photos"].apply(len)
X_train["num_features"] = X_train["features"].apply(len)
X_train["num_description_words"] = X_train["description"].apply(lambda x: len(x.split(" ")))
X_train["created"] = pd.to_datetime(X_train["created"])
X_train["created_year"] = X_train["created"].dt.year
X_train["created_month"] = X_train["created"].dt.month
X_train["created_day"] = X_train["created"].dt.day


In [ ]:
X_test["num_photos"] = X_test["photos"].apply(len)
X_test["num_features"] = X_test["features"].apply(len)
X_test["num_description_words"] = X_test["description"].apply(lambda x: len(x.split(" ")))
X_test["created"] = pd.to_datetime(X_test["created"])
X_test["created_year"] = X_test["created"].dt.year
X_test["created_month"] = X_test["created"].dt.month
X_test["created_day"] = X_test["created"].dt.day


In [ ]:
target_num_map = {'high':0, 'medium':1, 'low':2}
X_train["label"] = X_train["interest_level"].apply(lambda x: target_num_map[x])
X_train.head()

In [ ]:
y = X_train["label"]

X_train2, X_val, y_train2, y_val = train_test_split(X_train, y, test_size=0.3)

In [ ]:
CATEGORICAL_COLUMNS = ["building_id","description", "display_address", "street_address"]
#CATEGORICAL_COLUMNS = ["building_id","description", "display_address", "features", "street_address"]
CONTINUOUS_COLUMNS = ["bathrooms", "bedrooms", "latitude", "longitude", "price", "num_photos", "listing_id", 
                      "num_features", "num_description_words", "created_year", "created_month", "created_day"]
LABEL_COLUMN =["label"]
import tensorflow as tf

In [ ]:
def input_fn(df):
    #Creates a dictionary mapping from each continuous feature column name (k) to
    #the values of that column stored in a constant Tensor.
    continuous_cols = {k: tf.constant(df[k].values) for k in CONTINUOUS_COLUMNS}      
    #Creates a dictionary mapping from each categorical feature column name (k)
    #to the values of that column stored in a tf.SparseTensor.    
    categorical_cols = {k: tf.SparseTensor(indices=[[i, 0] for i in range(df[k].size)], values=df[k].values, dense_shape=[df[k].size, 1]) for k in CATEGORICAL_COLUMNS}
    # Merges the two dictionaries into one.
    feature_cols=continuous_cols
    feature_cols.update(categorical_cols)
    # Converts the label column into a constant Tensor.
    label = tf.constant(df[LABEL_COLUMN].values)
    #Returns the feature columns and the label.
    return feature_cols, label

def train_input_fn():
    return input_fn(X_train2)

def eval_input_fn():
    return input_fn(X_val)



In [ ]:
train_input_fn()
eval_input_fn()